In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [3]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756204215, 'localtime': '2025-08-26 03:30'}, 'current': {'last_updated_epoch': 1756204200, 'last_updated': '2025-08-26 03:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 13.0, 'windchill_f': 55.3, 'heatindex_c': 13.6, 'heatindex_f': 56.4, 'dewpoint_c': 13.0,

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

Enter API key for OpenAI:  ········


In [5]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today? 😊'

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [8]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco', 'search_depth': 'basic'}, 'id': 'call_wlIvEF9fpf1kp7FaYpr4xY9Q', 'type': 'tool_call'}]


In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [10]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [11]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_PL0vLit4unrqCOPIamJA2A9d)
 Call ID: call_PL0vLit4unrqCOPIamJA2A9d
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756207034, 'localtime': '2025-08-26 04:17'}, 'current': {'last_updated_epoch': 1756206900, 'last_updated': '2025-08-26 04:15', 'temp_c': 1

In [12]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_QEtSW9UfCdkZQwZw0ccCTmgW)
 Call ID: call_QEtSW9UfCdkZQwZw0ccCTmgW
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756207034, 'localtime': '2025-08-26 04:17'}, 'current': {'last_updated_epoch': 1756206900, 'last_updated': '2025-08-26 04:15', 'temp_c': 17.2, 'temp_f': 63.0, 'is

In [13]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| partly| cloudy| with| a| temperature| of| around| |16|.|7|°C| (|62|.|1|°F|).| The| wind| is| blowing| from| the| west|-s|outh|west| at| |6|.|5| mph| (|10|.|4| k|ph|),| and| humidity| is| |86|%.| It| feels| about| the| same| as| the| actual| temperature|.

|For| more| details|,| you| can| check| the| weather| site| here|.|

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [15]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [16]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello, Bob! How can I help you today?


In [17]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob! How can I assist you further?


In [18]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

You haven’t shared your name yet. Could you please tell me your name?
